In [7]:
import pandas as pd
import numpy as np
import random

data_raw = pd.read_csv("./data/example.csv", sep='|')
data_raw.head()

,ORDERKEY,PARTKEY,SUPPKEY,LINENUMBER,QUANTITY,EXTENDEDPRICE,DISCOUNT,TAX,RETURNFLAG,LINESTATUS,SHIPDATE,COMMITDATE,RECEIPTDATE,SHIPINSTRUCT,SHIPMODE
0,1,155190,7706,1,17,21168.23,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER_IN_PERSON,TRUCK
1,1,67310,7311,2,36,45983.16,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE_BACK_RETURN,MAIL
2,1,63700,3701,3,8,13309.60,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE_BACK_RETURN,REG_AIR
3,1,2132,4633,4,28,28955.64,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR
4,1,24027,1534,5,24,22824.48,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB


In [8]:
def generate_query(data, id1, id2, fname):
    n1 = data.columns[id1]
    n2 = data.columns[id2]
    col1 = data[n1]
    col2 = data[n2]
    flag1 = pd.api.types.is_numeric_dtype(col1)
    flag2 = pd.api.types.is_numeric_dtype(col2)
    dic1 = {}
    dic2 = {}

    if not flag1:
        col1 = col1.astype('category')
        dic1 = col1.drop_duplicates(keep='first').sort_values(ignore_index=True).to_dict()
        col1 = col1.cat.codes
    
    if not flag2:
        col2 = col2.astype('category')
        dic2 = col2.drop_duplicates(keep='first').sort_values(ignore_index=True).to_dict()
        col2 = col2.cat.codes

    sample = pd.concat([col1, col2], axis=1, ignore_index=True)
    sample.columns =[n1, n2]
    sample.to_csv(fname + "_data.csv", index=False)

    query = pd.DataFrame(columns=["lb1", "lb2", "ub1", "ub2", "GT"])

    max1 = col1.max()
    min1 = col1.min()
    max2 = col2.max()
    min2 = col2.min()

    GTs = []

    with open(fname + ".txt", 'w') as file:
        file.write("1\ntpch_example/lineitem.meta;0,1,2,3,4,5,6,7,8,9,10,11,12,13,14;;")
        for i in range(1000):
            x = random.random()
            y = random.random()
            lb1 = min1 + min(x, y) * (max1 - min1)
            ub1 = min1 + max(x, y) * (max1 - min1)
            if not flag1:
                lb1 = int(np.trunc(lb1))
                ub1 = int(np.trunc(ub1))

            x = random.random()
            y = random.random()
            lb2 = min2 + min(x, y) * (max2 - min2)
            ub2 = min2 + max(x, y) * (max2 - min2)
            if not flag2:
                lb2 = int(np.trunc(lb2))
                ub2 = int(np.trunc(ub2))

            GT = sample[(sample[n1] >= lb1) & (sample[n1] <= ub1) & (sample[n2] >= lb2) & (sample[n2] <= ub2)].shape[0]
            GTs.append(GT)

            query.loc[query.shape[0]] = [lb1, lb2, ub1, ub2, GT]
            
            if not flag1:
                lb1 = dic1[lb1]
                ub1 = dic1[ub1]
            
            if not flag2:
                lb2 = dic2[lb2]
                ub2 = dic2[ub2]

            file.write(f"{id1},{id2};{lb1},{lb2},{ub1},{ub2};")

        for i in range(999):
            file.write(f"{GTs[i]},")
        file.write(f"{GTs[999]};;")

    query.to_csv(fname + "_query.csv", index=False)

In [11]:
generate_query(data_raw, 4, 7, "test/test6")